### Notebook to create BLOCK-T91 for LUT in rotator angle

Created on: 2024-10-09

Author: Guillem Megias

In [ ]:
from lsst.ts.observing import ObservingBlock, ObservingScript 
from lsst.ts.aos.analysis import build_configuration_schema
import os

In [ ]:
current_path = os.getcwd()
block_number = 'T91'
program = "BLOCK-T91"
reason = "SITCOM-1491"
note = "LUT_rotator_angle"
constraints = []

### Define configuration schema

In [ ]:
# Define the configurable properties that we will use in the configuration schema
properties = {
    "run_filter": {
        "description": "ComCam filter to use.",
        "type": "string",
        "default": "r_03"
    },
    "run_azimuth": {
        "description": "Azimuth to use for the rotator angle sweep.",
        "type": "number",
        "default": 20
    },
    "run_elevation": {
        "description": "Elevation to use for the rotator angle sweep.",
        "type": "number",
        "default": 80
    }
}

# Build the configuration schema for BLOCK-404
configuration_schema = build_configuration_schema(block_number, properties)
print(configuration_schema)

### Define scripts and block

In [ ]:
comcam_triplets_script = ObservingScript(
    name="maintel/take_triplet_comcam.py",
    standard=True,
    parameters= dict(
        filter="$run_filter",
        program="$program",
        reason=reason,
        note=note
    )
)

stop_tracking_script = ObservingScript(
    name="maintel/stop_tracking.py",
    standard=True,
    parameters = dict()
)

In [ ]:
rotator_positions = [-75.0, -63.75, -42.5, -21.25, 0.0, 21.25, 42.5, 63.75, 75.0]
scripts = []

for rotator_angle in rotator_positions:
    track_target_script = ObservingScript(
        name="maintel/track_target.py",
        standard=True,
        parameters = dict(
            track_azel = dict(
                az = '$run_azimuth',
                el = '$run_elevation',
            ),
            rot_value = rotator_angle,
            rot_type = "PhysicalSky",
        )
    )

    scripts.append(track_target_script)
    scripts.append(comcam_triplets_script)
scripts.append(stop_tracking_script)

In [ ]:
block = ObservingBlock(
    name = program,
    program = program,
    configuration_schema=configuration_schema,
    scripts = scripts,
)

### Save configurable block

In [ ]:
block.model_dump_json(indent=2)

output_file_path = f'{current_path}/aos/ts_config_ocs/Scheduler/observing_blocks_maintel/AOS/LUTs/{program}.json'

with open(output_file_path, 'w') as file:
    file.write(block.model_dump_json(indent=2))